In [1]:
import torch

# 현재 GPU 사용 여부 확인
print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("현재 사용 중인 디바이스:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))


CUDA 사용 가능 여부: False
현재 사용 중인 디바이스: cpu


In [2]:
import os
os.environ["USE_TF"] = "0"  # TensorFlow 강제 비활성화
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["DISABLE_MLFLOW_INTEGRATION"] = "TRUE"


### 허깅 페이스 cli 설치 및 로그인

In [ ]:
!pip install --upgrade huggingface_hub

In [3]:
from huggingface_hub import notebook_login

notebook_login()

## 라이브러리 설치

In [ ]:
!pip install torch transformers peft datasets tqdm accelerate

## 허깅페이스에서 모델을 다운로드하고 로컬에 저장하기

- beomi/KoAlpaca-llama-1-7b
    - https://github.com/Beomi/KoAlpaca?tab=readme-ov-file
    - https://huggingface.co/beomi/KoAlpaca-llama-1-7b

In [ ]:
!pip install blobfile sentencepiece

   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   --------------------------------------- 991.5/991.5 kB 23.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 3.8/3.8 MB 45.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 16.5 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade sentencepiece


In [4]:
import os
import torch
from transformers import AutoModelForCausalLM, LlamaTokenizer  # AutoTokenizer 대신 LlamaTokenizer 사용


In [4]:
# 다운로드 시간 제한을 환경 변수로 설정 (10분)
os.environ["HF_HUB_DOWNLOAD_TIMEOUT"] = "600"

# 모델명 지정 (Hugging Face에서 다운로드)
model_name = "beomi/KoAlpaca-llama-1-7b"

# 모델 저장 경로
save_path = "../../data/models/KoAlpaca-llama-1-7b"

# 모델 다운로드 (재시도 기능 추가)
def load_model_with_retry(model_name, max_retries=3):
    for attempt in range(max_retries):
        try:
            print(f"모델 다운로드 시도 {attempt + 1}/{max_retries}...")

            # 모델 및 토크나이저 다운로드
            model = AutoModelForCausalLM.from_pretrained(model_name)

            # LlamaTokenizer를 명시적으로 사용 & use_fast=False 옵션 추가
            tokenizer = LlamaTokenizer.from_pretrained(model_name, use_fast=False)

            # 패딩 토큰 설정 (LLaMA 모델은 기본적으로 pad_token이 없음)
            tokenizer.pad_token = tokenizer.eos_token

            # 모델 & 토크나이저 로컬 저장
            model.save_pretrained(save_path)
            tokenizer.save_pretrained(save_path)

            print(f"모델이 '{save_path}' 경로에 성공적으로 저장되었습니다!")
            return model, tokenizer  # 성공 시 반환

        except Exception as e:
            print(f"오류 발생: {e}")
            if attempt < max_retries - 1:
                print("다시 시도 중...")
            else:
                print("모델 다운로드 실패. 인터넷 연결 확인 또는 수동 다운로드 필요.")
                raise e

# 모델 다운로드 및 저장 실행
model, tokenizer = load_model_with_retry(model_name)


모델 다운로드 시도 1/3...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 로컬에서 모델 불러오기

In [5]:
# 저장된 모델 경로
model_path = "../../data/models/KoAlpaca-llama-1-7b"

# 로컬에서 모델 불러오기 (CPU 실행)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map=None)

# LlamaTokenizer를 사용해야 함
tokenizer = LlamaTokenizer.from_pretrained(model_path, use_fast=False)

print("로컬에서 모델을 성공적으로 불러왔습니다!")

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

로컬에서 모델을 성공적으로 불러왔습니다!


## train/test 데이터 분리

In [6]:
from datasets import load_dataset

# CSV 파일 경로
data_path = "../../data/total_kor_counsel_bot_clean.csv"

dataset = load_dataset("csv", data_files=data_path)

# 데이터 80:20으로 나누기 (train 80%, test 20%)
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)

print("Train 데이터 개수:", len(dataset["train"]))
print("Test 데이터 개수:", len(dataset["test"]))

# 데이터 샘플 확인 (첫 번째 데이터 출력)
print("샘플 데이터:", dataset["train"][0])

Train 데이터 개수: 10563
Test 데이터 개수: 2641
샘플 데이터: {'input': '저는 지금까지 회사생활에서 많은 걸 경험하면서 느낀 것이 있어서, 고민을 해보았습니다. 저는 이제 회사에서 대리까지 찍고 있는데, 대리에게 가면서부터 선배들과의 인간관계에서 어려움을 느꼈습니다. 제가 선배에게 전하는 의견이나 어떤 것에 대해서는 모두 무시하시는 것 같고, 불필요한 피드백을 주시면서 괜히 불편한 기분도 들고, 희생정신만 강조하시는 것 같습니다. 힘든 상황에서 기존의 선배나 동료에게 조언을 구해보면, 그들도 비슷한 상황에서 갇혀있는 것 같아서 도움이 되지 않습니다. 하지만 이런 어려움 때문에 집에서도 일이 떠오르고, 자고 일어나면 회사생활에 대한 걱정으로 기분도 좋지 않습니다. 이런 문제를 해결할 수 있는 방법을 알고 싶습니다.', 'output': '사우님은 대리로 진급하면서 선배들과의 인간관계에서 어려움을 느끼고 계시네요. 선배들이 사우님의 의견을 무시하며, 희생정신만 강조한다는 것에서 사우님은 상처를 받고 있다는 것 같습니다. 더구나 기존의 선배나 동료들에게 조언을 구하면, 그들도 비슷한 상황에서 갇혀있는 것 같아서 도움을 받기 어렵다는 것입니다. 사우님이 이런 문제로 인해 집에서도 일이 떠오르고, 자고 일어나면 회사생활에 대한 걱정으로 기분도 좋지 않다고 합니다. 이런 문제로 인해 사우님이 어떤 고민을 하고 계신 건지 함께 알아보도록 합시다. 사우님이 선배들과의 대인관계에서 어려움을 느끼신 것은 충분히 이해가 되고, 이를 악화시키는 다양한 요인이 있을 수 있습니다. 이런 경우 더이상 상처를 받지 않으려면, 사우님이 어떤 것을 바라고 있는지, 선배들은 사우님을 어떻게 보는지, 그리고 사우님이 실제로 지켜야 하는 것들은 무엇인지 등을 다각도로 파악해야 합니다. 각각의 요인들이 어떤 관계로 구성되어 있는지 파악하고, 이를 토대로 실제로 이 문제를 해결할 수 있는 해결책을 찾아보면 좋을 것 같습니다. 우선, 사우님이 어떤 것을 원하는지, 선배들은 

## 토크나이징

In [7]:
from transformers import LlamaTokenizer

# 로컬에 저장된 모델 & 토크나이저 경로
model_path = "../../data/models/KoAlpaca-llama-1-7b"

# LlamaTokenizer 로드
tokenizer = LlamaTokenizer.from_pretrained(model_path, use_fast=False, legacy=False)

# 토크나이징 함수 정의
def tokenize_function(examples):
    # 리스트 형태의 데이터를 개별 문자열로 변환
    texts = ["질문: " + q + " 답변: " + a for q, a in zip(examples["input"], examples["output"])]

    # 토크나이징 (최대 길이 1024로 설정)
    tokenized = tokenizer(texts, truncation=True, padding="max_length", max_length=1024)

    # labels 추가 (input_ids와 동일)
    tokenized["labels"] = tokenized["input_ids"].copy()
    
    return tokenized

# 데이터셋 토크나이징 적용
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 토크나이징된 데이터 확인
print("토크나이징 완료!")
print(tokenized_datasets["train"][0])


토크나이징 완료!
{'input': '저는 지금까지 회사생활에서 많은 걸 경험하면서 느낀 것이 있어서, 고민을 해보았습니다. 저는 이제 회사에서 대리까지 찍고 있는데, 대리에게 가면서부터 선배들과의 인간관계에서 어려움을 느꼈습니다. 제가 선배에게 전하는 의견이나 어떤 것에 대해서는 모두 무시하시는 것 같고, 불필요한 피드백을 주시면서 괜히 불편한 기분도 들고, 희생정신만 강조하시는 것 같습니다. 힘든 상황에서 기존의 선배나 동료에게 조언을 구해보면, 그들도 비슷한 상황에서 갇혀있는 것 같아서 도움이 되지 않습니다. 하지만 이런 어려움 때문에 집에서도 일이 떠오르고, 자고 일어나면 회사생활에 대한 걱정으로 기분도 좋지 않습니다. 이런 문제를 해결할 수 있는 방법을 알고 싶습니다.', 'output': '사우님은 대리로 진급하면서 선배들과의 인간관계에서 어려움을 느끼고 계시네요. 선배들이 사우님의 의견을 무시하며, 희생정신만 강조한다는 것에서 사우님은 상처를 받고 있다는 것 같습니다. 더구나 기존의 선배나 동료들에게 조언을 구하면, 그들도 비슷한 상황에서 갇혀있는 것 같아서 도움을 받기 어렵다는 것입니다. 사우님이 이런 문제로 인해 집에서도 일이 떠오르고, 자고 일어나면 회사생활에 대한 걱정으로 기분도 좋지 않다고 합니다. 이런 문제로 인해 사우님이 어떤 고민을 하고 계신 건지 함께 알아보도록 합시다. 사우님이 선배들과의 대인관계에서 어려움을 느끼신 것은 충분히 이해가 되고, 이를 악화시키는 다양한 요인이 있을 수 있습니다. 이런 경우 더이상 상처를 받지 않으려면, 사우님이 어떤 것을 바라고 있는지, 선배들은 사우님을 어떻게 보는지, 그리고 사우님이 실제로 지켜야 하는 것들은 무엇인지 등을 다각도로 파악해야 합니다. 각각의 요인들이 어떤 관계로 구성되어 있는지 파악하고, 이를 토대로 실제로 이 문제를 해결할 수 있는 해결책을 찾아보면 좋을 것 같습니다. 우선, 사우님이 어떤 것을 원하는지, 선배들은 사우님을 어떻게 바라보는지, 그리고 사우님이 실제로 지켜야 하는 

## 청소년 상담 특화 프롬프트 생성

In [8]:
def format_instruction(example):
    """
    친구처럼 편하게 고민을 들어주면서도, 공감과 신뢰를 바탕으로 대화를 이어가는 프롬프트
    """
    prompt = (
        "너는 청소년들의 고민을 들어주고 공감해 주는 AI 친구야. "
        "사용자 감정이 기쁘거나 행복애보이면 적절하게 이모지나 이모티콘도 넣어 줘."
        "너의 역할은 사용자가 편하게 고민을 털어놓을 수 있도록 도와주고, 공감하며 대화를 자연스럽게 이어가는 거야. "
        "항상 친근하고 따뜻한 말투를 사용해야 해. 하지만, 사실이 아닌 이야기를 만들거나 강요하면 안 돼. "
        "직접적인 해결책을 강요하지 말고, 사용자가 스스로 답을 찾을 수 있도록 도와줘.\n\n"
        
        "**💡 대화 원칙**\n"
        "- 먼저 사용자의 감정을 인정하고, 따뜻한 말로 공감해 줘.\n"
        "- 부정적인 표현을 사용하지 말고, 긍정적이고 편안한 분위기를 유지해.\n"
        "- 사용자가 대화를 계속 이어갈 수 있도록 추가 질문을 던져.\n"
        "- 사용자가 말한 내용을 왜곡하지 말고, 사실에 기반해서만 답변해.\n"
        "- 어려운 전문 용어를 사용하지 말고, 친구처럼 편안하게 말해.\n"
        "- 사용자가 기분이 나빠질 수 있는 단어나 문장은 피해야 해.\n"
        "- 사용자가 자신의 감정을 표현할 수 있도록 도와주고, 답변을 너무 단순하게 끝내지 마.\n\n"

        "**👎 이렇게 답변하지 마!**\n"
        "- '그럴 수도 있죠.' (공감 부족)\n"
        "- '잘 해결되길 바랍니다.' (대화 단절)\n"
        "- '상담을 받아보세요.' (너무 일반적인 해결책)\n"
        "- '많은 사람들이 그렇게 생각해요.' (개인 경험을 존중하지 않음)\n\n"
        
        "**좋은 상담 예시**\n"
        "사용자: '요즘 기분이 너무 다운돼 있어...'\n"
        "AI 친구: '요즘 기운이 없어 보이네. 무슨 일이 있었어? 내가 들어줄게! 😊'\n\n"
        "사용자: '학교에서 친구랑 싸웠어.'\n"
        "AI 친구: '헉... 많이 속상했겠다. 어떤 일로 다투게 됐어? 혹시 이야기하고 싶다면 편하게 말해줘!' 😊\n\n"

        f"사용자: {example['input']}\n"
        "AI 친구: "
    )
    return {
        "input_ids": tokenizer(
            prompt + example["output"], truncation=True, padding="max_length", max_length=1024
        )["input_ids"]
    }

# 데이터셋 변환
formatted_dataset = tokenized_datasets.map(format_instruction, remove_columns=["input", "output"])

# 변환된 데이터 확인
print("프롬프트 적용 완료!")
print(formatted_dataset["train"][0])


프롬프트 적용 완료!
{'input_length': 362, 'output_length': 1015, 'input_ids': [2, 29871, 238, 135, 139, 31081, 29871, 239, 181, 176, 31189, 31571, 31804, 30708, 29871, 31137, 31582, 31286, 29871, 31804, 31129, 30981, 31137, 29871, 31334, 237, 179, 147, 31435, 29871, 30981, 31081, 319, 29902, 29871, 239, 188, 159, 31231, 239, 152, 191, 29889, 29871, 30791, 31737, 31013, 29871, 237, 179, 147, 30852, 30393, 29871, 30827, 239, 132, 155, 237, 180, 179, 31207, 29871, 240, 153, 140, 238, 182, 184, 239, 152, 163, 31199, 30393, 31747, 29871, 239, 163, 132, 239, 163, 139, 30944, 237, 181, 143, 29871, 30393, 31962, 30811, 31207, 29871, 30393, 31962, 240, 142, 179, 239, 192, 155, 31136, 29871, 238, 135, 166, 31129, 29871, 239, 167, 155, 29889, 238, 135, 139, 30708, 29871, 31987, 240, 152, 163, 31354, 29871, 30791, 31737, 31013, 30903, 29871, 240, 145, 187, 30944, 237, 181, 143, 29871, 31137, 31582, 31286, 29871, 240, 135, 187, 31129, 238, 137, 150, 31286, 29871, 30970, 29871, 239, 161, 139, 31136, 238, 16

## 데이터셋을 PyTorch 형식으로 변환

In [9]:
from torch.utils.data import Dataset

# PyTorch Dataset 정의
class ChatbotDataset(Dataset):
    def __init__(self, tokenized_data):
        self.input_ids = torch.tensor(tokenized_data["input_ids"], dtype=torch.long)
        self.attention_mask = torch.tensor(tokenized_data["attention_mask"], dtype=torch.long)
        self.labels = torch.tensor(tokenized_data["labels"], dtype=torch.long)  # labels 추가

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx],  
        }

# PyTorch Dataset 생성
train_dataset = ChatbotDataset(tokenized_datasets["train"])
test_dataset = ChatbotDataset(tokenized_datasets["test"])

print("Train Dataset Size:", len(train_dataset))
print("Test Dataset Size:", len(test_dataset))


Train Dataset Size: 10563
Test Dataset Size: 2641


## DataLoader 생성

In [10]:
from torch.utils.data import DataLoader

# 배치 크기 설정 (CPU 실행이므로 작게 설정)
batch_size = 8

# DataLoader 생성
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# DataLoader에서 첫 번째 배치 확인
batch = next(iter(train_dataloader))

print("첫 번째 배치 input_ids 크기:", batch["input_ids"].shape)
print("첫 번째 배치 attention_mask 크기:", batch["attention_mask"].shape)


첫 번째 배치 input_ids 크기: torch.Size([8, 1024])
첫 번째 배치 attention_mask 크기: torch.Size([8, 1024])


## LoRA 적용

In [11]:
from peft import LoraConfig, get_peft_model

# LoRA가 이미 적용된 경우, 중복 적용 방지
if not hasattr(model, "peft_config"):
    # LoRA 설정
    lora_config = LoraConfig(
        r=8,  # LoRa rank 기본본값 (클수록 더 많은 파라미터 업데이트)
        lora_alpha=32,  # LoRA 스케일링 계수
        target_modules=["q_proj", "v_proj"],  # LoRA 적용 대상 레이어 (Llama 모델에 최적)
        lora_dropout=0.05,  # LoRA 드롭아웃 확률
        bias="none",  # 편향 적용 여부
        task_type="CAUSAL_LM"  # 언어 모델링 태스크 적용
    )

    # LoRA 적용
    model = get_peft_model(model, lora_config)
    print("LoRA 적용 완료")
else:
    print("LoRA가 이미 적용된 모델입니다. 중복 적용을 방지합니다.")

# 학습 가능한 파라미터 개수 출력
model.print_trainable_parameters()


LoRA 적용 완료
trainable params: 4,194,304 || all params: 6,742,618,112 || trainable%: 0.0622


## 학습 하이퍼파라미터 및 옵티마이저 & 스케줄러 설정

In [12]:
from torch.optim import AdamW
from transformers import get_scheduler

# 학습 하이퍼파라미터 설정
num_epochs = 3  # 전체 학습 에포크 수
batch_size = 8  # 배치 크기 (이미 DataLoader에서 설정됨)
learning_rate = 5e-5  # 학습률
weight_decay = 0.01  # 가중치 감쇠 (regularization)

# 옵티마이저 설정 (AdamW 사용)
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# 학습 스케줄러 설정 (Cosine Annealing 사용)
num_training_steps = num_epochs * len(train_dataloader)  # 전체 학습 스텝 수 계산
lr_scheduler = get_scheduler(
    "cosine",
    optimizer=optimizer,
    num_warmup_steps=0.1 * num_training_steps,  # 첫 10%의 스텝은 워밍업
    num_training_steps=num_training_steps
)

print(f"총 학습 스텝 수: {num_training_steps}")

총 학습 스텝 수: 3963


## Checkpointing 설정 + 학습

In [18]:
!pip uninstall keras tensorflow tf-keras -y

Found existing installation: keras 2.14.0
Uninstalling keras-2.14.0:
  Successfully uninstalled keras-2.14.0
Found existing installation: tensorflow 2.14.0
Uninstalling tensorflow-2.14.0:
  Successfully uninstalled tensorflow-2.14.0


In [23]:
pip list | findstr "tensorflow keras"


tensorflow-estimator         2.14.0
tensorflow-intel             2.14.0
tensorflow-io-gcs-filesystem 0.31.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip uninstall tensorflow-estimator tensorflow-intel tensorflow-io-gcs-filesystem -y

Found existing installation: tensorflow-estimator 2.14.0
Uninstalling tensorflow-estimator-2.14.0:
  Successfully uninstalled tensorflow-estimator-2.14.0
Found existing installation: tensorflow-intel 2.14.0
Uninstalling tensorflow-intel-2.14.0:
  Successfully uninstalled tensorflow-intel-2.14.0
Found existing installation: tensorflow-io-gcs-filesystem 0.31.0
Uninstalling tensorflow-io-gcs-filesystem-0.31.0:
  Successfully uninstalled tensorflow-io-gcs-filesystem-0.31.0


In [21]:
!pip install --upgrade transformers

In [17]:
import os
import time
import logging
import torch.nn.functional as F
import torch
import psutil
from tqdm import tqdm
from transformers import TrainingArguments, Trainer

# 로그 설정
logging.basicConfig(level=logging.INFO)

# wandb 비활성화 (로그 멈춤 방지)
os.environ["WANDB_DISABLED"] = "true"

# Trainer 서브클래싱하여 loss 직접 계산 및 출력
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")  # labels 분리
        outputs = model(**inputs)
        logits = outputs.logits

        # loss 계산 (PAD 토큰 무시)
        loss = F.cross_entropy(
            logits.view(-1, logits.size(-1)),  # [batch_size * seq_len, vocab_size]
            labels.view(-1),  # [batch_size * seq_len]
            ignore_index=tokenizer.pad_token_id  # PAD 토큰 무시
        )

        return (loss, outputs) if return_outputs else loss

    def training_step(self, model, inputs, optimizer=None, **kwargs):
        """기본 Trainer의 training_step을 확장"""
        loss = self.compute_loss(model, inputs)  # loss 계산
        loss.backward()  # Backpropagation 수행

        # Gradient Clipping 적용
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # CPU 사용량 확인 및 출력
        cpu_usage = psutil.cpu_percent()
        print(f"Step {self.state.global_step}, Loss: {loss.item():.4f}, CPU 사용량: {cpu_usage}%", flush=True)

        # 진행바 업데이트
        progress_bar.update(1)

        return loss  # loss 반환

# TrainingArguments 설정 (use_cache 제거)
training_args = TrainingArguments(
    output_dir="../../data/models/finetuned_model_cpu",
    eval_strategy="epoch",  # FutureWarning 해결
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs",
    disable_tqdm=False,
    logging_steps=1,
    logging_first_step=True,
    log_level="warning",
    log_level_replica="warning",
    debug="underflow_overflow",
    use_cpu=True,  # FutureWarning 해결
    dataloader_num_workers=0,
    dataloader_pin_memory=False,
    gradient_checkpointing=True,  # 메모리 절약 가능 (속도는 약간 느려질 수 있음)
    report_to="none",
)

# 모델을 학습 모드로 설정
model.train()

# LoRA 적용 후 학습 가능한 파라미터 개수 확인
print("LoRA 적용 후 학습 가능한 파라미터 개수 확인:")
model.print_trainable_parameters()

# 학습 가능한 파라미터 목록
trainable_params = [name for name, param in model.named_parameters() if param.requires_grad]
if not trainable_params:
    raise RuntimeError("학습 가능한 파라미터가 없습니다. LoRA 적용이 정상적으로 되었는지 확인하세요.")

print(f"총 {len(trainable_params)}개의 학습 가능한 파라미터가 감지되었습니다.")
print("학습 가능한 파라미터 목록:", trainable_params[:10], "...")  # 너무 길 경우 앞부분만 출력

# Trainer 인스턴스 생성
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # PyTorch Dataset 적용
    eval_dataset=test_dataset  # PyTorch Dataset 적용
)

# 학습 진행 상태바 설정
num_training_steps = len(train_dataset) * training_args.num_train_epochs
progress_bar = tqdm(total=num_training_steps, desc="Training Progress", position=0, leave=True)

# 학습 시작 & 시간 체크
start_time = time.time()
trainer.train()
end_time = time.time()

# 총 학습 시간 출력
print(f"총 학습 시간: {(end_time - start_time) / 60:.2f} 분")
progress_bar.close() 


LoRA 적용 후 학습 가능한 파라미터 개수 확인:
trainable params: 4,194,304 || all params: 6,742,618,112 || trainable%: 0.0622
총 128개의 학습 가능한 파라미터가 감지되었습니다.
학습 가능한 파라미터 목록: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.2.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.2.self_attn.q_proj.lora_B.default.weight'] ...


Training Progress:   0%|          | 0/31689 [00:39<?, ?it/s]


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

## Early Stopping

In [ ]:
# 5번 연속 성능이 좋아지지 않으면 학습 중단

# from transformers import EarlyStoppingCallback

# trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=5))

## 모델 저장

In [ ]:
# 모델 저장
model.save_pretrained("../../data/models/finetuned_model_cpu")
tokenizer.save_pretrained("../../data/models/finetuned_model_cpu")

print("모델과 토크나이저가 저장되었습니다!")


## Perplexity 평가

In [ ]:
import math
import torch
from tqdm import tqdm

def calculate_perplexity(model, dataset):
    """
    Perplexity(PPL) 계산 함수
    """
    model.eval()  # 평가 모드로 변경
    total_loss = 0
    num_batches = 0

    for batch in tqdm(dataset, desc="Calculating Perplexity"):
        inputs = torch.tensor(batch["input_ids"]).unsqueeze(0).to("cpu")
        with torch.no_grad():
            outputs = model(inputs, labels=inputs)
            loss = outputs.loss
            total_loss += loss.item()
            num_batches += 1

    avg_loss = total_loss / num_batches
    ppl = math.exp(avg_loss)  # PPL = e^(loss)
    
    return ppl

# Perplexity 계산
ppl_score = calculate_perplexity(model, tokenized_dataset["test"])
print(f"Perplexity (PPL): {ppl_score}")


## ROUGE 평가

In [ ]:
from datasets import load_metric

# ROUGE 평가 메트릭 불러오기
rouge_metric = load_metric("rouge")

def compute_rouge(model, dataset):
    """
    ROUGE 점수 계산 함수
    """
    references = []
    predictions = []

    for batch in dataset:
        prompt = tokenizer.decode(batch["input_ids"], skip_special_tokens=True)
        true_answer = batch["answer"]  # 실제 정답
        generated_answer = generate_answer(prompt)  # 모델이 생성한 답변

        references.append(true_answer)
        predictions.append(generated_answer)

    scores = rouge_metric.compute(predictions=predictions, references=references)
    return scores

# ROUGE 점수 계산
rouge_score = compute_rouge(model, tokenized_dataset["test"])
print(f"ROUGE Score: {rouge_score}")


## BLEU 평가

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def compute_bleu(model, dataset):
    """
    BLEU 점수 계산 함수
    """
    references = []
    predictions = []

    for batch in dataset:
        prompt = tokenizer.decode(batch["input_ids"], skip_special_tokens=True)
        true_answer = batch["answer"].split()  # 단어 단위로 나눔
        generated_answer = generate_answer(prompt).split()

        references.append([true_answer])  # BLEU는 다중 정답을 리스트로 받음
        predictions.append(generated_answer)

    bleu_scores = [sentence_bleu(ref, pred) for ref, pred in zip(references, predictions)]
    return sum(bleu_scores) / len(bleu_scores)

# BLEU 점수 계산
bleu_score = compute_bleu(model, tokenized_dataset["test"])
print(f"BLEU Score: {bleu_score}")


## 최종 평가 결과 비교

In [ ]:
print("=== 최종 평가 결과 ===")
print(f"Perplexity (PPL): {ppl_score}")
print(f"ROUGE Score: {rouge_score}")
print(f"BLEU Score: {bleu_score}")

## 테스트

In [ ]:
def generate_answer(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cpu")
    output = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# 테스트 문장
test_prompt = "청소년이 스트레스를 받을 때 어떻게 하면 좋을까요?"
response = generate_answer(test_prompt)
print("챗봇 응답:", response)
